In [ ]:
import plotly.express as px

from core.sql_utils import *
from core.pandas_utils import *
from core.plt_utils import *
from core.caching_utils import cache_result
from transform.fleet_info.main import fleet_info
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

In [ ]:
@cache_result("data_cache/tesla_tss.parquet", on='local_storage')
def get_tss() -> DF:
    return ProcessedTimeSeries("tesla")

@cache_result("data_cache/tesla_charges.parquet", on='local_storage')
def get_charges() -> DF:
    return get_tss().pipe(compute_charges)

def compute_charges(tss:DF) -> DF:
    return (
        tss
        .drop(columns=fleet_info.drop(columns=["vin"]).columns)
        .merge(fleet_info, on="vin", how="left")
        .query("trimmed_in_charge")
        .groupby(["vin", "trimmed_in_charge_idx"])
        .agg(
            energy_added=pd.NamedAgg("charge_energy_added", series_start_end_diff),
            soc_diff=pd.NamedAgg("soc", series_start_end_diff),
            soc_start=pd.NamedAgg("soc", "first"),
            soc_end=pd.NamedAgg("soc", "last"),
            inside_temp=pd.NamedAgg("inside_temp", "mean"),
            outside_temp=pd.NamedAgg("outside_temp", "mean"),
            capacity=pd.NamedAgg("capacity", "first"),
            odometer=pd.NamedAgg("odometer", "first"),
            fast_charger_type=pd.NamedAgg("fast_charger_type", "first"),
            size=pd.NamedAgg("soc", "size"),
            model=pd.NamedAgg("model", "first"),
            version=pd.NamedAgg("version", "first"),
            date=pd.NamedAgg("date", "first"),
            charge_rate=pd.NamedAgg("charge_rate", "median"),
            fast_charger_present=pd.NamedAgg("fast_charger_present", "median"),
            charge_current_request=pd.NamedAgg("charge_current_request", "median"),
            tesla_code=pd.NamedAgg("tesla_code", "first"),
            battery_heater=pd.NamedAgg("battery_heater", "median"),
        )
        .reset_index(drop=False)
        .eval("soh = energy_added / (soc_diff / 100.0 * capacity)")
        .eval("model_version = model + version")
    )

@cache_result("data_cache/{vin}_tesla_ts.parquet", on='local_storage', path_params=["vin"])
def get_target_ts(vin:str) -> DF:
    return get_tss().query("vin == @vin")

In [ ]:
TARGET_VIN = "5YJ3E7EB7KF474436"
ts = get_tss(force_update=True).query("vin == @TARGET_VIN")

In [ ]:
ts.re

In [ ]:
charges = compute_charges(ts)

In [ ]:
charges.describe()

In [ ]:
all_charges = get_charges()

In [ ]:
all_vins = all_charges["vin"].unique()
fleet_info.query("make == 'tesla'").eval("vin in @all_vins").value_counts(dropna=False)

In [ ]:
all_vins.size

In [ ]:
12932 + 652

In [ ]:
fleet_info.query("make == 'tesla' & vin.notna()").size

In [ ]:
fleet_info.query('vin == @TARGET_VIN')